# 1. Importación de librerías

In [2]:
# Librerías para manejo de datos
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import numpy as np
np.random.seed(3301)
import pandas as pd
# Para preparar los datos
from sklearn.preprocessing import LabelEncoder
# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 
# Para usar KNN como clasificador
from sklearn.neighbors import KNeighborsClassifier
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
#Librerías para la visualización
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns 
from sklearn import tree

# 2. Limpieza y Perfilamiento de Datos

In [ ]:
# Completar

# 3. Algoritmo KNN (K-Nearest Neighbors)

In [ ]:
# Se selecciona la variable objetivo, en este caso "Popularity_label".
Y = df_tracks_m['popularity_label']
# Del conjunto de datos se elimina la variable "Popularity_label"
X = df_tracks_m.drop(['popularity_label'], axis=1)

In [ ]:
# Dividir los datos en entrenamiento y test
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
# neigh = KNeighborsClassifier(n_neighbors=3)
# neigh = neigh.fit(X_train, Y_train)

In [ ]:
# y_pred = neigh.predict(X_test)

In [ ]:
# Se genera la matriz de confusión
# confusion_matrix(Y_test, y_pred)

In [ ]:
# Se puede visualizar la matriz de confusión
# plot_confusion_matrix(neigh, X_test, Y_test)  
# plt.show()

In [ ]:
# Mostrar reporte de clasificación
# print(classification_report(Y_test, y_pred))